In [1]:
input_video = r'Demo_video\demo.mp4'  # 輸入影片路徑


In [2]:
import cv2
import numpy as np
input_align_video = input_video[:-4]+'_align.mp4'

def find_common_region(frames):
    """ 計算所有對齊影格的共同可視區域，避免黑色邊框 """
    h, w = frames[0].shape[:2]
    mask = np.ones((h, w), dtype=np.uint8) * 255  #白色遮罩 (255 代表可用區域)

    for frame in frames:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)  #找出非黑色區域
        mask = cv2.bitwise_and(mask, binary)  #取得所有影格的交集區域

    #找出最小邊界框 (bounding box)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    x, y, w, h = cv2.boundingRect(contours[0])  #取得最大共視範圍

    return x, y, w, h

def extract_fixed_frames_and_align(input_video, output_video, target_frames=240):
    #讀取影片
    cap = cv2.VideoCapture(input_video)
    
    #確保影片能夠正確開啟
    if not cap.isOpened():
        print("Error: 無法開啟影片")
        return
    
    #取得影片基本資訊
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_interval = max(1, total_frames // target_frames)

    frames = []
    frame_idx = 0
    frame_count = 0

    #讀取影格
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame_count >= target_frames:
            break
        if frame_idx % sample_interval == 0:
            frames.append(frame)
            frame_count += 1
        frame_idx += 1

    cap.release()

    if len(frames) == 0:
        raise ValueError("未能讀取任何影格，請檢查影片路徑與格式。")

    frames = np.array(frames, dtype=np.uint8)  #儲存影格

    #RANSAC + Homography 影像對齊
    def align_images(frames):
        aligned_frames = []
        base_frame = frames[0]  #以第一幀為基準
        orb = cv2.ORB_create()

        for frame in frames:
            kp1, des1 = orb.detectAndCompute(cv2.cvtColor(base_frame, cv2.COLOR_BGR2GRAY), None)
            kp2, des2 = orb.detectAndCompute(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), None)
            
            if des1 is None or des2 is None or len(des1) == 0 or len(des2) == 0:
                aligned_frames.append(frame)
                continue
            
            bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
            matches = bf.match(des1, des2)
            matches = sorted(matches, key=lambda x: x.distance)
            
            if len(matches) > 10:
                src_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
                dst_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
                H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
                aligned_frame = cv2.warpPerspective(frame, H, (frame.shape[1], frame.shape[0]))
                aligned_frames.append(aligned_frame)
            else:
                aligned_frames.append(frame)

        return np.array(aligned_frames, dtype=np.uint8)

    aligned_frames = align_images(frames)

    #計算最小可視區域 (去除黑邊)
    x, y, w, h = find_common_region(aligned_frames)
    cropped_frames = [frame[y:y+h, x:x+w] for frame in aligned_frames]  #裁剪所有影格

    #輸出影片
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = 30  #設定輸出影片 FPS
    out = cv2.VideoWriter(output_video, fourcc, fps, (w, h))

    for frame in cropped_frames:
        out.write(frame)

    out.release()
    print(f"✅ 影片儲存成功！輸出路徑: {output_video}")


extract_fixed_frames_and_align(input_video, input_align_video, target_frames=240)


✅ 影片儲存成功！輸出路徑: Demo_video\demo_align.mp4
